<a href="https://colab.research.google.com/github/LeomaxFilho/Operational-Research/blob/main/Pratica_Destinacao_Produtos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula prática: Destinação de Produtos à Fabricas
<sup>Exemplo do capítulo 8.3 do livro `Introdução à Pesquisa Operacional` de `Hillier e Lieberman`.</sup>

## Exercício 1

### Descrição do problema
A Cia. Produtos Melhores decidiu iniciar a produção de quatro produtos novos usando três fábricas que, no momento, têm excesso de capacidade produtiva. Os produtos requerem um esforço de produção comparável por unidade, de modo que a capacidade produtiva disponível das fábricas seja medida pelo número de unidades de qualquer produto que possa ser produzido diariamente, conforme dado na tabela 1. A tabela 2 fornece a taxa de produção diária necessária para atender às vendas projetadas. Cada fábrica é capaz de produzir qualquer um desses produtos, exceto a Fábrica 2, que não pode fabricar o produto 3. Os custos variáveis por unidade de cada produto diferem de fábrica para fábrica, conforme a tabela 3.

Tabela 1: Capacidade de produção disponível, por unidade do produto

| | Fábrica 1 | Fábrica 2 | Fábrica 3 |
|:---|:---:|:---:|:---:|
| Capacidade disponível | 75 | 75 | 45 |

Tabela 2: Demanda de produção diária

| | Produto 1 | Produto 2 | Produto 3 | Produto 4 |
|:---|:---:|:---:|:---:|:---:|
| Demanda | 20 | 30 | 30 | 40 |

Tabela 3: Custo unitário por produto

| | Produto 1 | Produto 2 | Produto 3 | Produto 4 |
|:---|:---:|:---:|:---:|:---:|
| Fábrica 1 | 41 | 27 | 28 | 24 |
| Fábrica 2 | 40 | 29 | - | 23 |
| Fábrica 3 | 37 | 30 | 27 | 21 |

A gerência precisa tomar uma decisão sobre como dividir a fabricação dos produtos entre as fábricas, e decidiu permitir a divisão da produção de um mesmo produto em mais de uma fábrica.

Escreva um modelo para determinar quais fábricas produzirão quais produtos e a que custo total.

### Resolução

In [ ]:
# instalação e importação do pacote mip
!pip install mip
from mip import *

# funcões usadas posteriormente:

# resolve o modelo e mostra os valores das variáveis
def solve(model):
  status = model.optimize()

  print("Status = ", status)
  print(f"Solution value  = {model.objective_value:.2f}\n")

  print("Solution:")
  for v in model.vars:
      print(f"{v.name} = {v.x:.2f}")


# salva modelo em arquivo lp, e mostra o conteúdo
def save(model, filename):
  model.write(filename) # salva modelo em arquivo
  with open(filename, "r") as f: # lê e exibe conteúdo do arquivo
    print(f.read())

In [ ]:
model = # ... criar modelo
save(model, "model.lp")

In [ ]:
solve(model)

## Exercício 2

Refaça o modelo anterior, dessa vez impedindo a divisão de produtos entre fábricas. Ou seja, cada produto deve ser integralmente produzido em uma única fábrica, para eliminar custos ocultos associados à divisão da produção. A cada fábrica deve ser destinado pelo menos um produto.

### Resolução

In [ ]:
model = # ... criar modelo

save(model, "model2.lp")

In [ ]:
solve(model)